In [ ]:
# https://heegyukim.medium.com/huggingface-koelectra%EB%A1%9C-nsmc-%EA%B0%90%EC%84%B1%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0-1a23a0c704af

+ 목적 : Pre-train된 모형(transformer)을 통해, 리뷰 값을 예측 및 정확도를 평가함

+ (Precision, Recall)수행모델 : 정밀도, 재현율
  + [ ] (0.59, 0.86) koelectra-small-v3-nsmc
  + [ ] (0.62, 0.71) bert-base-multilingual-cased-nsmc
  + [ ] (이용  불가) klue-roberta-base-nli
  + [ ] (이용  불가) snunlp/KR-FinBert-SC

+ 이슈사항
  + [ ] koelectra-small-v3-nsmc : 비꼬는 내용은 인지 불가
  + [ ] bert-base-multilingual-cased-nsmc : 비꼬는 내용은 인지 불가
  + [ ] klue-roberta-base-nli
  

In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
from transformers import BertTokenizer
from sentence_transformers import SentenceTransformer

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
# df_ = pd.read_excel('./20220620_kurly_review.xlsx', 0)
# df_ = df_[['REVIEW', 'SCORE', ]]

In [4]:
df_maeil2 = pd.read_excel('./20220615_review_data.xlsx','Sheet1')
df_maeil2 = df_maeil2[['REVIEW', 'SCORE', ]]

In [5]:
df = df_maeil2.copy()

df['REVIEW'] = df['REVIEW'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
df['REVIEW'] = df['REVIEW'].replace("\n", " ")

# 긍종/부정으로 나누기
df['label'] = np.select([df.SCORE > 3], [1], default=0)

# 중복 제거
# df['SCORE'].nunique(), df['REVIEW'].nunique(), df['label'].nunique()
# df.drop_duplicates(subset=['REVIEW'], inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
df.SCORE[:10]

0    2
1    2
2    2
3    2
4    2
5    2
6    2
7    2
8    2
9    2
Name: SCORE, dtype: int64

### 1.koelectra-small-v3-nsmc

In [6]:
from transformers import pipeline
classifier = pipeline( "sentiment-analysis", model="daekeun-ml/koelectra-small-v3-nsmc" )

In [21]:
# test
t1 = classifier("ㅜㅜ완모에서 혼합으로가려는데  즈이애가 안먹네요ㅜ")
list(t1[0].values())[0], list(t1[0].values())[1] 

('1', 0.8639047741889954)

In [12]:
t1

[{'label': '0', 'score': 0.997473418712616}]

In [20]:
# test2
text = list(df.REVIEW.tail(10))
score_list = []

for i in range(len(text)):
    try:
        t1 = classifier(text[i])
        gubun, score = list(t1[0].values())[0], list(t1[0].values())[1] 
        if gubun == '0':
            print(gubun, round((1-score)*5, 2), text[i])
        elif gubun == '1':
            print(gubun, round((score)*5, 2), text[i])
        else:
            print(gubun, round(score*5, 2), text[i])
    except:
        pass

0 0.01 흠 그다지 좋지는 않네요 애매모호
0 0.03 ㅜㅜ 아이가안먹네요
0 0.03 ㅜㅜ 좁쌀두드레기 약하게올라와서 밥먹이는거 중단하고 우유만먹였는데 늘먹던거라 리뉴얼되었다고 이럴줄은 생각도 못해서ㅜㅜ 완전 두드레기 엄청올라오고 병원가서 알레르기검사한다고 피까지 뽑았네요 하맛  단 편이에요용해  보통이에요
0 0.07 ㅜㅜ맘큐 좋아서 제휴업체꺼도 구입했는데 맘큐 제품만 좋은가바여 전 사진에 나와있는 길쭉한 팩으로 시켰는데 더 저렴한 사각팩으로 보내어요 인터넷에찾아봐도 가격다르고 여기서도 분명 가격 달랐는데 제휴업체는 같다고 계속 말하니 제휴업체 말로는 이게 리뉴얼돼서 없어다는데 재구매는 안하려고 알겟어요 하고 대답하고 걍 먹여요 그 이후에 마켓컬리에서 팔길래 산 길쭉한 팩은 유통기한도 더 길어요뭔가 이상 맘큐 개선 부탁드려요 애정하는 몰인데ㅜㅜㅜㅜㅜㅜ
0 0.01 ㅜㅜ맛이별론가봐요 방송보고 추천해서샀는데 아이가안먹어요 두박스나샀는데ㅜㅜ
0 0.06 ㅜㅜ아이가안먹어요 제가먹어야겠네요
1 4.32 ㅜㅜ완모에서 혼합으로가려는데  즈이애가 안먹네요ㅜ
1 4.96 ㅠㅠ 너무 달아요 건강 챙기려고 주문했는데 당뇨병 걸릴거같아요 너뮤 달아
0 0.01 ㅠㅠ 살때는 유통기한 안써있었는데ㅠㅠ 뭔가 사기당한기분ㅠㅠ 큰사이즈랑 두개 구매했는데 이거 먼저 하루에 두개씩 먹여야겠어요 항상 구매하는데 이번에는 좀 실망입니다
0 0.01 ㅠㅠ 안먹어요 한입먹어보더니 보기만해도 싫더네요 우쉬워요   ㅠㅠ


In [95]:
text = list(df.REVIEW)
score_list = []

for i in tqdm(range(len(text))):
    try:
        t1 = classifier(text[i])
        gubun, score = list(t1[0].values())[0], list(t1[0].values())[1] 
        if gubun == '0':
            score_list.append(1-score) 
        else:
            score_list.append(score)
    except:
        score_list.append(0)

100%|████████████████████████████████████████████████████████████████████████████| 14623/14623 [08:30<00:00, 28.63it/s]


In [96]:
df1 = df.copy()
df1['label_model'] = score_list
df1['label_model'] = round(df1['label_model']*5,0)

In [97]:
# Excel 저장
# date2 = datetime.today().strftime('%Y%m%d')
# excel_name = '_review_data2.xlsx'
# df1.to_excel(excel_name, sheet_name= 'Sheet1' )

In [99]:
df1 = df.copy()

df1['SCORE_model'] = score_list
df1['SCORE_model'] = round(df1['SCORE_model']*5,0)

df1['label_model'] = np.select([df1.SCORE_model > 3], [1], default=0)

TN = df1.groupby(['label','label_model'])['SCORE'].count().reset_index()[0:1]['SCORE'].values
FP = df1.groupby(['label','label_model'])['SCORE'].count().reset_index()[1:2]['SCORE'].values
FN = df1.groupby(['label','label_model'])['SCORE'].count().reset_index()[2:3]['SCORE'].values
TP = df1.groupby(['label','label_model'])['SCORE'].count().reset_index()[3:4]['SCORE'].values

model1_precision = TP/(TP+FP)
model1_recall    = TP/(TP+FN)

In [104]:
print('< 긍정/부정 분류에 대한 결과 비교 >')
print('precision : {}'.format(round(model1_precision[0],5)))
print('recall    : {}'.format(round(model1_recall[0],5)))

< 긍정/부정 분류에 대한 결과 비교 >
precision : 0.59278
recall    : 0.86783


### 2. sangrimlee/bert-base-multilingual-cased-nsmc

In [134]:
df_

,REVIEW,SCORE
0,아기가잘먹아주네요~~~,NaN
1,처음에는 못먹더니 이제 적응했는지 잘먹네오,NaN
2,아기용으로 자주사요,NaN
3,자주 사 먹이는데 아기가 좋아해요,NaN
4,집에 떨어지지않게 늘 재구매,NaN
...,...,...
245,차스타 소스 굿이에요.,NaN
246,상하키친~ 파스타만 삶아서 소스 부어주고 편해요,NaN
247,아기가 잘 먹어주면 좋겠어요,NaN
248,맛도 궁금하고 먹여보고 싶어서 구매,NaN


In [13]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="sangrimlee/bert-base-multilingual-cased-nsmc")

In [140]:
# test1
t1 = classifier("옆에 틈이 벌어졌어요 다 샜어요 어떡하나요?")
list(t1[0].values())[0], (list(t1[0].values())[1] )

('Default', 0.8754841685295105)

In [135]:
# test2
text = list(df_.REVIEW.tail(10))
score_list = []

for i in range(len(text)):
    try:
        t1 = classifier(text[i])
        gubun, score = list(t1[0].values())[0], list(t1[0].values())[1] 
        if gubun == 'negative':
            print(gubun, round((1-score)*5, 0), text[i])
        else:
            print(gubun, round(score*5, 0), text[i])
    except:
        pass

Default 5.0 제일 잘 먹는 카레라 자주 시켜요
Default 4.0 상하키친 포크 카레 맛있어요
Default 5.0 맛있고 건강한 재료로 만들어서 자주 사먹으려구요
Default 5.0 뻑뻑해서 우유넣고. . 치즈넣고. . 
식구들은 잘먹었지만 저하고는 안맞았어요. .
사진은 다른 제품입니다;;
Default 5.0 맛있는데 좀 가격이 있는 편인듯
재구매 고민좀
Default 5.0 차스타 소스 굿이에요.
Default 5.0 상하키친~ 파스타만 삶아서 소스 부어주고 편해요
Default 5.0 아기가 잘 먹어주면 좋겠어요
Default 5.0 맛도 궁금하고 먹여보고 싶어서 구매
Default 5.0 만들어보기전에 참고할겸 샀습니다


In [18]:
text = list(df.REVIEW)
score_list = []

for i in tqdm(range(len(text))):
    try:
        t1 = classifier(text[i])
        gubun, score = list(t1[0].values())[0], list(t1[0].values())[1] 
        if gubun == 'negative':
            score_list.append(1-score) 
        else:
            score_list.append(score)
    except:
        score_list.append(0)

100%|████████████████████████████████████████████████████████████████████████████| 14623/14623 [23:34<00:00, 10.34it/s]


In [24]:
df2['SCORE_model'] = score_list
df2['SCORE_model'] = round(df2['SCORE_model']*5,0)

df2['label_model'] = np.select([df2.SCORE_model > 3], [1], default=0)

TN = df2.groupby(['label','label_model'])['SCORE'].count().reset_index()[0:1]['SCORE'].values
FP = df2.groupby(['label','label_model'])['SCORE'].count().reset_index()[1:2]['SCORE'].values
FN = df2.groupby(['label','label_model'])['SCORE'].count().reset_index()[2:3]['SCORE'].values
TP = df2.groupby(['label','label_model'])['SCORE'].count().reset_index()[3:4]['SCORE'].values

model2_precision = TP/(TP+FP)
model2_recall    = TP/(TP+FN)

In [71]:
print('< 긍정/부정 분류에 대한 결과 비교 >')
print('precision : {}'.format(round(model2_precision[0],5)))
print('recall    : {}'.format(round(model2_recall[0],5)))

긍정/부정 분류에 대한 결과 비교
precision : 0.61859
recall    : 0.71333


In [70]:
# Excel 저장
# date2 = datetime.today().strftime('%Y%m%d')
# excel_name = '_review_data3.xlsx'
# df.to_excel(excel_name, sheet_name= 'Sheet1' )

In [106]:
model_nm ="Huffon/klue-roberta-base-nli"
classifier = pipeline("sentiment-analysis", model=model_nm)

Downloading:   0%|          | 0.00/926 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [110]:
# test1
t1 = classifier("흠 그다지 좋지는 않네요 애매모호")
list(t1[0].values())[0], list(t1[0].values())[1] 

('ENTAILMENT', 0.9895753860473633)

In [109]:
t1

[{'label': 'NEUTRAL', 'score': 0.9959205389022827}]

In [111]:
# test2
text = list(df.REVIEW.tail(10))
score_list = []

for i in range(len(text)):
    try:
        t1 = classifier(text[i])
        gubun, score = list(t1[0].values())[0], list(t1[0].values())[1] 
        if gubun == 'negative':
            print(gubun, round((1-score)*5, 0), text[i])
        else:
            print(gubun, round(score*5, 0), text[i])
    except:
        pass

ENTAILMENT 5.0 흠 그다지 좋지는 않네요 애매모호
NEUTRAL 4.0 ㅜㅜ 아이가안먹네요
NEUTRAL 5.0 ㅜㅜ 좁쌀두드레기 약하게올라와서 밥먹이는거 중단하고 우유만먹였는데 늘먹던거라 리뉴얼되었다고 이럴줄은 생각도 못해서ㅜㅜ 완전 두드레기 엄청올라오고 병원가서 알레르기검사한다고 피까지 뽑았네요 하맛  단 편이에요용해  보통이에요
NEUTRAL 5.0 ㅜㅜ맘큐 좋아서 제휴업체꺼도 구입했는데 맘큐 제품만 좋은가바여 전 사진에 나와있는 길쭉한 팩으로 시켰는데 더 저렴한 사각팩으로 보내어요 인터넷에찾아봐도 가격다르고 여기서도 분명 가격 달랐는데 제휴업체는 같다고 계속 말하니 제휴업체 말로는 이게 리뉴얼돼서 없어다는데 재구매는 안하려고 알겟어요 하고 대답하고 걍 먹여요 그 이후에 마켓컬리에서 팔길래 산 길쭉한 팩은 유통기한도 더 길어요뭔가 이상 맘큐 개선 부탁드려요 애정하는 몰인데ㅜㅜㅜㅜㅜㅜ
NEUTRAL 5.0 ㅜㅜ맛이별론가봐요 방송보고 추천해서샀는데 아이가안먹어요 두박스나샀는데ㅜㅜ
NEUTRAL 5.0 ㅜㅜ아이가안먹어요 제가먹어야겠네요
NEUTRAL 5.0 ㅜㅜ완모에서 혼합으로가려는데  즈이애가 안먹네요ㅜ
NEUTRAL 3.0 ㅠㅠ 너무 달아요 건강 챙기려고 주문했는데 당뇨병 걸릴거같아요 너뮤 달아
NEUTRAL 5.0 ㅠㅠ 살때는 유통기한 안써있었는데ㅠㅠ 뭔가 사기당한기분ㅠㅠ 큰사이즈랑 두개 구매했는데 이거 먼저 하루에 두개씩 먹여야겠어요 항상 구매하는데 이번에는 좀 실망입니다
CONTRADICTION 5.0 ㅠㅠ 안먹어요 한입먹어보더니 보기만해도 싫더네요 우쉬워요   ㅠㅠ


In [112]:
model_nm ="snunlp/KR-FinBert-SC"
classifier = pipeline("sentiment-analysis", model=model_nm)

In [113]:
# test1
t1 = classifier("흠 그다지 좋지는 않네요 애매모호")
list(t1[0].values())[0], list(t1[0].values())[1] 

('neutral', 0.8784013390541077)

In [115]:
# test2
text = list(df.REVIEW.tail(10))
score_list = []

for i in range(len(text)):
    try:
        t1 = classifier(text[i])
        gubun, score = list(t1[0].values())[0], list(t1[0].values())[1] 
        if gubun == 'negative':
            print(gubun, round((1-score)*5, 0), text[i])
        else:
            print(gubun, round(score*5, 0), text[i])
    except:
        pass

neutral 4.0 흠 그다지 좋지는 않네요 애매모호
neutral 5.0 ㅜㅜ 아이가안먹네요
neutral 5.0 ㅜㅜ 좁쌀두드레기 약하게올라와서 밥먹이는거 중단하고 우유만먹였는데 늘먹던거라 리뉴얼되었다고 이럴줄은 생각도 못해서ㅜㅜ 완전 두드레기 엄청올라오고 병원가서 알레르기검사한다고 피까지 뽑았네요 하맛  단 편이에요용해  보통이에요
neutral 5.0 ㅜㅜ맘큐 좋아서 제휴업체꺼도 구입했는데 맘큐 제품만 좋은가바여 전 사진에 나와있는 길쭉한 팩으로 시켰는데 더 저렴한 사각팩으로 보내어요 인터넷에찾아봐도 가격다르고 여기서도 분명 가격 달랐는데 제휴업체는 같다고 계속 말하니 제휴업체 말로는 이게 리뉴얼돼서 없어다는데 재구매는 안하려고 알겟어요 하고 대답하고 걍 먹여요 그 이후에 마켓컬리에서 팔길래 산 길쭉한 팩은 유통기한도 더 길어요뭔가 이상 맘큐 개선 부탁드려요 애정하는 몰인데ㅜㅜㅜㅜㅜㅜ
neutral 5.0 ㅜㅜ맛이별론가봐요 방송보고 추천해서샀는데 아이가안먹어요 두박스나샀는데ㅜㅜ
neutral 5.0 ㅜㅜ아이가안먹어요 제가먹어야겠네요
neutral 5.0 ㅜㅜ완모에서 혼합으로가려는데  즈이애가 안먹네요ㅜ
neutral 5.0 ㅠㅠ 너무 달아요 건강 챙기려고 주문했는데 당뇨병 걸릴거같아요 너뮤 달아
neutral 5.0 ㅠㅠ 살때는 유통기한 안써있었는데ㅠㅠ 뭔가 사기당한기분ㅠㅠ 큰사이즈랑 두개 구매했는데 이거 먼저 하루에 두개씩 먹여야겠어요 항상 구매하는데 이번에는 좀 실망입니다
neutral 5.0 ㅠㅠ 안먹어요 한입먹어보더니 보기만해도 싫더네요 우쉬워요   ㅠㅠ


In [116]:
model_nm = "saattrupdan/verdict-classifier"
classifier = pipeline("sentiment-analysis", model=model_nm)

Downloading:   0%|          | 0.00/934 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [117]:
# test1
t1 = classifier("흠 그다지 좋지는 않네요 애매모호")
list(t1[0].values())[0], list(t1[0].values())[1] 

('misinformation', 0.9964070916175842)

In [119]:
# test2
text = list(df.REVIEW.tail(10))
score_list = []

for i in range(len(text)):
    try:
        t1 = classifier(text[i])
        gubun, score = list(t1[0].values())[0], list(t1[0].values())[1] 
        if gubun == 'misinformation':
            print(gubun, round((1-score)*5, 0), text[i])
        else:
            print(gubun, round(score*5, 0), text[i])
    except:
        pass

misinformation 0.0 흠 그다지 좋지는 않네요 애매모호
misinformation 0.0 ㅜㅜ 아이가안먹네요
other 5.0 ㅜㅜ 좁쌀두드레기 약하게올라와서 밥먹이는거 중단하고 우유만먹였는데 늘먹던거라 리뉴얼되었다고 이럴줄은 생각도 못해서ㅜㅜ 완전 두드레기 엄청올라오고 병원가서 알레르기검사한다고 피까지 뽑았네요 하맛  단 편이에요용해  보통이에요
other 5.0 ㅜㅜ맘큐 좋아서 제휴업체꺼도 구입했는데 맘큐 제품만 좋은가바여 전 사진에 나와있는 길쭉한 팩으로 시켰는데 더 저렴한 사각팩으로 보내어요 인터넷에찾아봐도 가격다르고 여기서도 분명 가격 달랐는데 제휴업체는 같다고 계속 말하니 제휴업체 말로는 이게 리뉴얼돼서 없어다는데 재구매는 안하려고 알겟어요 하고 대답하고 걍 먹여요 그 이후에 마켓컬리에서 팔길래 산 길쭉한 팩은 유통기한도 더 길어요뭔가 이상 맘큐 개선 부탁드려요 애정하는 몰인데ㅜㅜㅜㅜㅜㅜ
other 5.0 ㅜㅜ맛이별론가봐요 방송보고 추천해서샀는데 아이가안먹어요 두박스나샀는데ㅜㅜ
misinformation 0.0 ㅜㅜ아이가안먹어요 제가먹어야겠네요
misinformation 0.0 ㅜㅜ완모에서 혼합으로가려는데  즈이애가 안먹네요ㅜ
misinformation 0.0 ㅠㅠ 너무 달아요 건강 챙기려고 주문했는데 당뇨병 걸릴거같아요 너뮤 달아
other 5.0 ㅠㅠ 살때는 유통기한 안써있었는데ㅠㅠ 뭔가 사기당한기분ㅠㅠ 큰사이즈랑 두개 구매했는데 이거 먼저 하루에 두개씩 먹여야겠어요 항상 구매하는데 이번에는 좀 실망입니다
other 5.0 ㅠㅠ 안먹어요 한입먹어보더니 보기만해도 싫더네요 우쉬워요   ㅠㅠ


In [120]:
model_nm = "jason9693/SoongsilBERT-base-beep"
classifier = pipeline("sentiment-analysis", model=model_nm)

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/375M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/339k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [121]:
# test1
t1 = classifier("흠 그다지 좋지는 않네요 애매모호")
list(t1[0].values())[0], list(t1[0].values())[1] 

('none', 0.9931035041809082)

In [123]:
# test2
text = list(df.REVIEW.head(10))
score_list = []

for i in range(len(text)):
    try:
        t1 = classifier(text[i])
        gubun, score = list(t1[0].values())[0], list(t1[0].values())[1] 
        if gubun == 'misinformation':
            print(gubun, round((1-score)*5, 0), text[i])
        else:
            print(gubun, round(score*5, 0), text[i])
    except:
        pass

none 5.0 그냥 쏘쏘 왜 비싼지 몰겠음
none 5.0 땅콩 들어있어요 상품 설명 보시면 고한  견과류 캐슈넛이 들어있다고 되어 있고 땅콩이 있다는 설명이 없어서 샀는네 땅콩이 들어있어요 제가 땅콩 알려지거 좀 있어서 땅콩 든건 안먹는데 광고에 베리류랑 캐슈넛만 나와 있어서 샀다가 낭패봤습니다그래뇰라만으로도 달달한데 화이트초코까지 있어서 매우답니다 재구매 의사는 없습니다
none 5.0 아침대용으로 운동전에 먹을 꺼예요
none 5.0 일단고농축이라 믿고 잘먹어부겠습니다
none 5.0 맛나요
none 5.0 집앞까지 무거운 우유들 배송에 감사요
none 5.0 부드럽고 맛있어요 흰우유 안먹는 살 아이 초코우유 잘먹네요 근데 날짜가 짧긴하네요
none 5.0 맛좋아요잘먹ㄹ게요
none 5.0 맛도 괜찮고 좋아요
none 5.0 맛은 찐한것같은데 달아요


In [124]:
model_nm ='jason9693/KcELECTRA-small-v2022-apeach'
classifier = pipeline("sentiment-analysis", model=model_nm)

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/63.4M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [125]:
# test1
t1 = classifier("흠 그다지 좋지는 않네요 애매모호")
list(t1[0].values())[0], list(t1[0].values())[1] 

('Default', 0.9445276856422424)

In [128]:
# test2
text = list(df.REVIEW.tail(10))
score_list = []

for i in range(len(text)):
    try:
        t1 = classifier(text[i])
        gubun, score = list(t1[0].values())[0], list(t1[0].values())[1] 
        if gubun == 'Default':
            print(gubun, round((1-score)*5, 0), text[i])
        else:
            print(gubun, round(score*5, 0), text[i])
    except:
        pass

Default 0.0 흠 그다지 좋지는 않네요 애매모호
Default 0.0 ㅜㅜ 아이가안먹네요
Default 1.0 ㅜㅜ 좁쌀두드레기 약하게올라와서 밥먹이는거 중단하고 우유만먹였는데 늘먹던거라 리뉴얼되었다고 이럴줄은 생각도 못해서ㅜㅜ 완전 두드레기 엄청올라오고 병원가서 알레르기검사한다고 피까지 뽑았네요 하맛  단 편이에요용해  보통이에요
Default 0.0 ㅜㅜ맘큐 좋아서 제휴업체꺼도 구입했는데 맘큐 제품만 좋은가바여 전 사진에 나와있는 길쭉한 팩으로 시켰는데 더 저렴한 사각팩으로 보내어요 인터넷에찾아봐도 가격다르고 여기서도 분명 가격 달랐는데 제휴업체는 같다고 계속 말하니 제휴업체 말로는 이게 리뉴얼돼서 없어다는데 재구매는 안하려고 알겟어요 하고 대답하고 걍 먹여요 그 이후에 마켓컬리에서 팔길래 산 길쭉한 팩은 유통기한도 더 길어요뭔가 이상 맘큐 개선 부탁드려요 애정하는 몰인데ㅜㅜㅜㅜㅜㅜ
Default 0.0 ㅜㅜ맛이별론가봐요 방송보고 추천해서샀는데 아이가안먹어요 두박스나샀는데ㅜㅜ
Default 0.0 ㅜㅜ아이가안먹어요 제가먹어야겠네요
Default 1.0 ㅜㅜ완모에서 혼합으로가려는데  즈이애가 안먹네요ㅜ
Default 1.0 ㅠㅠ 너무 달아요 건강 챙기려고 주문했는데 당뇨병 걸릴거같아요 너뮤 달아
Default 0.0 ㅠㅠ 살때는 유통기한 안써있었는데ㅠㅠ 뭔가 사기당한기분ㅠㅠ 큰사이즈랑 두개 구매했는데 이거 먼저 하루에 두개씩 먹여야겠어요 항상 구매하는데 이번에는 좀 실망입니다
Default 1.0 ㅠㅠ 안먹어요 한입먹어보더니 보기만해도 싫더네요 우쉬워요   ㅠㅠ
